<a href="https://colab.research.google.com/github/rshamsini/COVID-19detection/blob/main/web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np #mathematical operations over arrays
import pandas as pd #a powerful data analysis and manipulation library for Python
import os
import glob #glob module is used to retrieve files/pathnames matching a specified pattern.
from pathlib import Path

In [ ]:
import torch
from IPython.display import Image, clear_output
import PIL
from PIL import Image

In [ ]:
from sklearn.externals import joblib

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Deployment of model using Flask


In [ ]:
#pip install dependencies

!pip install flask-ngrok

In [ ]:
#load the svm model that is saved in the storage

svm = joblib.load("/content/drive/MyDrive/Optum/model.pkl")

In [ ]:
#static folder contains css,js
#tempate folder contains index.html

import os
os.mkdir('templates')
os.mkdir('static')

In [ ]:
#index.html

index='''
<html lang="en">

<head>
    <meta charset="UTF-8">
    <title>AI Demo</title>
    <link href="https://cdn.bootcss.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet">
    <script src="https://cdn.bootcss.com/popper.js/1.12.9/umd/popper.min.js"></script>
    <script src="https://cdn.bootcss.com/jquery/3.3.1/jquery.min.js"></script>
    <script src="https://cdn.bootcss.com/bootstrap/4.0.0/js/bootstrap.min.js"></script>
    <script src="https://ajax.googleapis.com/ajax/libs/angularjs/1.6.9/angular.min.js"></script> 
    <link href="./static/css/main.css" rel="stylesheet">      
</head>

<body>
    <nav class="navbar navbar-dark bg-dark">
        <div class="container">
            <a class="navbar-brand" href="#">COVID-19 DETECTION</a>
        </div>
    </nav>
    <div class="container">
        <div id="content" style="margin-top:2em">{% block content %}{% endblock %}</div>
    </div>

<div class="new">
    <form id="upload-file" method="post" enctype="multipart/form-data">
        <label for="imageUpload" class="upload-label">
            Upload a file...
        </label>
        <input type="file" name="file" id="imageUpload" accept=".png, .jpg, .jpeg">
    </form>
    

    <div class="image-section" style="display:none;">
        <div class="img-preview">
            <div id="imagePreview">
            </div>
        </div>
        <div>
            <button type="button" class="btn btn-primary btn-lg " id="btn-predict">Predict!</button>
        </div>
    </div>


    <div class="loader" style="display:none;"></div>

    <h3 id="result">
        <span> </span>
    </h3>

    

</div>
</body>

<footer>
    <script src="./static/js/main.js" type="text/javascript"></script>    
</footer>

</html>
'''
file=open('templates/index.html','w')
file.write(index)
file.close()

In [ ]:


os.mkdir('static/css')

In [ ]:
#main.css

main='''
.img-preview {
    width: 256px;
    height: 256px;
    position: relative;
    border: 5px solid #F8F8F8;
    box-shadow: 0px 2px 4px 0px rgba(0, 0, 0, 0.1);
    margin-top: 1em;
    margin-bottom: 1em;   
     display: block;
     margin-left: auto;
     margin-right: auto; 
}

.new{
    margin-left: 10px;
    justify-content: center;
    text-align: center;
}
.navbar-brand{
  text-align: center;
}

.img-preview>div {
    width: 100%;
    height: 100%;
    background-size: 256px 256px;
    background-repeat: no-repeat;
    background-position: center;
}

input[type="file"] {
    display: none;
}

.upload-label{
    display: inline-block;
    padding: 12px 30px;
    background: #39D2B4;
    color: #fff;
    font-size: 1em;
    transition: all .4s;
    cursor: pointer;
}

.upload-label:hover{
    background: #34495E;
    color: #39D2B4;
}

.loader {
    border: 8px solid #f3f3f3; /* Light grey */
    border-top: 8px solid #3498db; /* Blue */
    border-radius: 50%;
    width: 50px;
    height: 50px;
    animation: spin 1s linear infinite;
    margin:0 auto;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
'''
file=open('static/css/main.css','w')
file.write(main)
file.close()

In [ ]:
os.mkdir('static/js')

In [ ]:
#main.js

main='''
$(document).ready(function () {
    // Init
    $('.image-section').hide();
    $('.loader').hide();
    $('#result').hide();

    // Upload Preview

    function readURL(input) {
        if (input.files && input.files[0]) {
            var reader = new FileReader();
            reader.onload = function (e) {
                $('#imagePreview').css('background-image', 'url(' + e.target.result + ')');
                $('#imagePreview').hide();
                $('#imagePreview').fadeIn(650);
            }
            reader.readAsDataURL(input.files[0]);
        }
    }
    $("#imageUpload").change(function () {
        $('.image-section').show();
        $('#btn-predict').show();
        $('#result').text('');
        $('#result').hide();
        readURL(this);
    });

    // Predict
    $('#btn-predict').click(function () {
        var form_data = new FormData($('#upload-file')[0]);

        // Show loading animation
        $(this).hide();
        $('.loader').show();

        // Make prediction by calling api /predict
        $.ajax({
            type: 'POST',
            url: '/predict',
            data: form_data,
            contentType: false,
            cache: false,
            processData: false,
            async: true,
            success: function (data) {
                // Get and display the result
                $('.loader').hide();
                $('#result').fadeIn(600);
                $('#result').text(' Result:  '+data);
                $('#imagePreview').css('background-image', 'url('+ e.target.result +')');
                console.log('Success!');
            },
        });
    });

});


'''
file=open('static/js/main.js','w')
file.write(main)
file.close()

In [ ]:
os.mkdir('uploads')

In [ ]:
%pip install -qr /content/drive/MyDrive/Optum/yolov5/requirements.txt  # install dependencies

In [ ]:
#dependencies

import torch
from IPython.display import Image, clear_output  # to display images

In [ ]:
#yolov5 setup

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (CPU)


In [ ]:
#flask dependencies

from flask_ngrok import run_with_ngrok
import json
import requests
from flask import request
from flask import Flask, redirect, url_for, request, render_template,Response
from werkzeug.utils import secure_filename
import random

In [ ]:
X_pred=[0.71875,0.425781,0.316406,0.558594]

In [ ]:
#sample code for prediction using classification model

#convert the pred list into 2-D np array

X_pred=np.array(X_pred)
X_pred = np.reshape(X_pred, (1, 4))
print(X_pred)

#predict the result using svm model

prediction=svm.predict(X_pred)
print(prediction)

[[0.71875  0.425781 0.316406 0.558594]]
[1]


# Predict Image function


In [ ]:
#class 0-->Negative for Pneumonia 
#class 1-->Typical Appearance 
#class 2-->Indeterminate Appearance 
#class 3-->Atypical Appearance

def predictImage(filename):

  result=['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']

  img_file=filename
  tf_name=filename[-16:]
  
  #tf_name[:-4] -> name of the label text file

  fname=tf_name[:-4]
  n = random.randint(0,22)
  fname=fname+str(n)
  
  #fname -> name of the folder where the result of test image is stored

  #predict the bounding box values
  #call detect.py , pass the weights,image path,result storage path

  output=!python '/content/drive/MyDrive/Optum/yolov5/detect.py' --weights '/content/drive/MyDrive/Optum/yolov5/runs/train/exp8/weights/best.pt' --source {img_file}  --img 640 --conf 0.281 --project 'test' --name {fname} --save-txt
  print(output)

  #path of the text file
  
  txt_file="test/"+ fname + "/labels/"+tf_name[:-4]+".txt"
  
  if os.path.isfile(txt_file):
    
    #read labels from the text file
    
    f = open(txt_file, "r")
    res=f.readline()
    res=res.rstrip('\n')
    res=res[2:]
    X_pred=res.split(" ")
    
    #store labels in a list

    for i in range(4):
      X_pred[i]=float(X_pred[i])
    
    #convert it into 2-D np array

    X_pred=np.array(X_pred)
    X_pred = np.reshape(X_pred, (1, 4))
    
  else:
    #labels for class none

    X_pred=[0.0,0.0,0.0,0.0]

    #convert it into 2-D np array  

    X_pred=np.array(X_pred)
    X_pred = np.reshape(X_pred, (1, 4))
  
  #predict result using the svm model

  prediction=svm.predict(X_pred)
  
  #print the result

  print(prediction)

  #return the class name of the result

  return result[prediction[0]]


# Flask App


In [ ]:
app=Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname('contents')
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = predictImage(file_path)
        
        return preds
    return None

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d9ead98fc42e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Aug/2021 07:00:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 07:00:04] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 07:00:04] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 07:00:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Aug/2021 07:01:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 07:01:36] "POST /predict HTTP/1.1" 200 -


["\x1b\x1bdetect: \x1bweights=['/content/drive/MyDrive/Optum/yolov5/runs/train/exp8/weights/best.pt'], source=uploads/8925b615fc62.png, imgsz=640, conf_thres=0.281, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=test, name=8925b615fc6210, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False", '\x1b\x1brequirements:\x1b /content/requirements.txt not found, check failed.', 'YOLOv5 🚀 v5.0-319-g63a1971 torch 1.9.0+cu102 CPU', '', 'Fusing layers... ', '/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)', '  return torch.max_pool2d(input, kerne

127.0.0.1 - - [09/Aug/2021 07:02:27] "POST /predict HTTP/1.1" 200 -


["\x1b\x1bdetect: \x1bweights=['/content/drive/MyDrive/Optum/yolov5/runs/train/exp8/weights/best.pt'], source=uploads/8866f708d338.png, imgsz=640, conf_thres=0.281, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=test, name=8866f708d3384, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False", '\x1b\x1brequirements:\x1b /content/requirements.txt not found, check failed.', 'YOLOv5 🚀 v5.0-319-g63a1971 torch 1.9.0+cu102 CPU', '', 'Fusing layers... ', '/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)', '  return torch.max_pool2d(input, kernel